In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ccnu_datatape_32x8x8_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_32x8x8_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train).cache(), to_tf(valid).cache()

In [3]:
model = keras.Sequential([
    layers.Input(shape=(32, 8, 8, 3)),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv3D(4, (2, 2, 2), (1, 2, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv3D(2, (2, 2, 2), (1, 2, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,)),
    layers.AvgPool3D((1, 2, 2)),
    layers.Conv3D(1, 1, 1),
    layers.Flatten(),
], name='NoobHeart')
model.compile(optimizer='adam', loss='mae')
print(f'Flops per frame:{get_flops(model, input_sig=[tf.TensorSpec([1, 32, 8, 8, 3])])/32:.0f}')
model.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:5790
Model: "NoobHeart"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization (LayerNo (None, 32, 8, 8, 3)       64        
_________________________________________________________________
conv3d (Conv3D)              (None, 32, 4, 4, 4)       100       
_________________________________________________________________
layer_normalization_1 (Layer (None, 32, 4, 4, 4)       64        
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 32, 2, 2, 2)       66        
_________________________________________________________________
layer_normalization_2 (Layer (None, 32, 2, 2, 2)       64        
_________________________________________________________________
average_pooling3d (AveragePo (None, 32, 1, 1, 2)       0         
_____________

In [4]:
model.fit(train.shuffle(2000).batch(128), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/NoobHeart_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
model.load_weights('../weights/NoobHeart_CCNU.h5')

Epoch 1/10
1189/1189 [==============================] - 66s 55ms/step - loss: 0.7734 - val_loss: 0.7431

Epoch 00001: val_loss improved from inf to 0.74308, saving model to ../weights\NoobHeart_CCNU.h5
Epoch 2/10
1189/1189 [==============================] - 52s 44ms/step - loss: 0.7382 - val_loss: 0.7364

Epoch 00002: val_loss improved from 0.74308 to 0.73641, saving model to ../weights\NoobHeart_CCNU.h5
Epoch 3/10
1189/1189 [==============================] - 52s 44ms/step - loss: 0.7301 - val_loss: 0.7143

Epoch 00003: val_loss improved from 0.73641 to 0.71432, saving model to ../weights\NoobHeart_CCNU.h5
Epoch 4/10
1189/1189 [==============================] - 52s 44ms/step - loss: 0.7217 - val_loss: 0.7065

Epoch 00004: val_loss improved from 0.71432 to 0.70648, saving model to ../weights\NoobHeart_CCNU.h5
Epoch 5/10
1189/1189 [==============================] - 51s 43ms/step - loss: 0.7202 - val_loss: 0.7041

Epoch 00005: val_loss improved from 0.70648 to 0.70410, saving model to ../

In [26]:
model.load_weights('../weights/NoobHeart.h5')

In [12]:
eval_on_dataset(test_set_CCNU, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_CCNU.h5')
get_metrics('../results/NoobHeart_CCNU_CCNU.h5',)

100%|██████████| 179/179 [00:18<00:00,  9.57it/s]


{'Sliding window': {'MAE': 1.785, 'RMSE': 5.845, 'R': 0.83193},
 'Whole video': {'MAE': 0.849, 'RMSE': 2.764, 'R': 0.95744}}

In [11]:
eval_on_dataset(test_set_CCNU_rPPG, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_CCNU_rPPG.h5')
get_metrics('../results/NoobHeart_CCNU_CCNU_rPPG.h5',)

100%|██████████| 54/54 [00:04<00:00, 13.18it/s]


{'Sliding window': {'MAE': 1.569, 'RMSE': 4.705, 'R': 0.88329},
 'Whole video': {'MAE': 0.754, 'RMSE': 1.476, 'R': 0.98773}}

In [10]:
eval_on_dataset(test_set_PURE, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_PURE.h5')
get_metrics('../results/NoobHeart_CCNU_PURE.h5',)

100%|██████████| 59/59 [00:02<00:00, 21.88it/s]


{'Sliding window': {'MAE': 0.765, 'RMSE': 2.447, 'R': 0.99443},
 'Whole video': {'MAE': 0.447, 'RMSE': 0.698, 'R': 0.99962}}

In [9]:
eval_on_dataset(test_set_UBFC_rPPG2, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_UBFC.h5')
get_metrics('../results/NoobHeart_CCNU_UBFC.h5',)

100%|██████████| 42/42 [00:01<00:00, 22.99it/s]


{'Sliding window': {'MAE': 1.777, 'RMSE': 4.324, 'R': 0.9701},
 'Whole video': {'MAE': 1.143, 'RMSE': 1.693, 'R': 0.99637}}